In [50]:
!pip install -q python-dotenv langchain langchain-community tiktoken duckduckgo-search beautifulsoup4 openai langchain-openai pymongo

# Agentic RAG
Normally, when we use RAG (Retrieval-Augmented Generation), we provide it with a single collection containing embeddings, and we perform a vector search on that collection. However, what if we have multiple collections covering different domains and need to perform vector searches across them based on the user's query?

In such scenarios, Agentic RAG becomes invaluable. It can intelligently search across multiple collections, perform internet searches when required, or even generate answers from its own knowledge base. This makes it an integral solution for handling diverse queries in a seamless and efficient manner.

In [12]:
import os
import json
import random
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.tools import DuckDuckGoSearchRun
import dotenv

dotenv.load_dotenv()


from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

url_to_name_map = {
    "https://lilianweng.github.io/posts/2023-06-23-agent/": "Agent_Post",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/": "Prompt_Engineering_Post",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/": "Adv_Attack_LLM_Post",
}


docs = {}
for url in url_to_name_map:
    try:
        loader = WebBaseLoader(url)
        docs[url] = loader.load()
    except Exception as e:
        print(f"Error loading {url}: {e}")


text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, chunk_overlap=50
)

doc_splits = {}
for url, doc in docs.items():
    doc_splits[url] = text_splitter.split_documents(doc)


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [48]:
import pymongo
from pymongo import MongoClient
from pymongo.operations import SearchIndexModel
import json
import random

# Connect to MongoDB Atlas
uri = "mongodb+srv://<USERNAME>:<PASSWORD>@cluster0.pbxvpvn.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0"
client = MongoClient(uri)
db = client["db"]


# Function to add data to the collection with embedding
def upload_data_to_collection(collection_name, data):
    try:
        collection = db[collection_name]
        embedding = embeddings.embed_query(data)  # Generate embedding
        
        # Insert document with embedding
        collection.insert_one({
            "text": data,
            "embedding": embedding
        })
        print(f"Data uploaded to collection '{collection}' successfully.")
        return json.dumps({"Message": "Data uploaded successfully"})
    except Exception as e:
        print(f"Error uploading data to {collection}: {e}")
        return json.dumps({"Error": str(e)})

# Function to create a vector search index on the collection
def create_vector_index(collection_name, num_dimensions):
    collection = db[collection_name]
    
    index_model = SearchIndexModel(
        definition={
            "fields": [
                {
                    "type": "vector",
                    "path": "embedding",
                    "similarity": "dotProduct",
                    "numDimensions": num_dimensions
                }
            ]
        },
        name="vector_index",
        type="vectorSearch"
    )
    collection.create_search_index(model=index_model)
    print(f"Vector search index created for collection '{collection}'.")

# Function to perform vector similarity search
def search_db(collection_name, input_query):
    collection = db[collection_name]
    print(f"Searching in collection")
    try:        
        query_embedding = embeddings.embed_query(input_query)  # Generate embedding for query

        # MongoDB vector search pipeline
        pipeline = [
            {
                "$vectorSearch": {
                    "index": "vector_index",
                    "queryVector": query_embedding,
                    "path": "embedding",
                    "exact": True,
                    "limit": 5
                }
            },
            {
                "$project": {
                    "_id": 0,
                    "text": 1,
                    "score": {
                        "$meta": "vectorSearchScore"
                    }
                }
            }
        ]

        # Execute the pipeline
        results = collection.aggregate(pipeline)
        result_docs = [{"text": res["text"], "score": res["score"]} for res in results]
        return json.dumps({"Data": result_docs})
    
    except Exception as e:
        print(f"Error searching in collection {collection}: {e}")
        return json.dumps({"Error": str(e)})
    
DuckDuckGo = DuckDuckGoSearchRun()
def Internet_search(input):
    print("\n\nSearching Internet \n\n")
    
    res = DuckDuckGo.run(input)
    return res



In [15]:
#Upload data to saperate collections for each URL
for url, splits in doc_splits.items():
    collection_name = url_to_name_map[url]  
    for chunk in splits:
        upload_data_to_collection(collection_name, chunk.page_content)


Data uploaded to collection 'Collection(Database(MongoClient(host=['ac-blk0g1e-shard-00-01.pbxvpvn.mongodb.net:27017', 'ac-blk0g1e-shard-00-02.pbxvpvn.mongodb.net:27017', 'ac-blk0g1e-shard-00-00.pbxvpvn.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='Cluster0', authsource='admin', replicaset='atlas-ue4svv-shard-0', tls=True), 'db'), 'Agent_Post')' successfully.
Data uploaded to collection 'Collection(Database(MongoClient(host=['ac-blk0g1e-shard-00-01.pbxvpvn.mongodb.net:27017', 'ac-blk0g1e-shard-00-02.pbxvpvn.mongodb.net:27017', 'ac-blk0g1e-shard-00-00.pbxvpvn.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='Cluster0', authsource='admin', replicaset='atlas-ue4svv-shard-0', tls=True), 'db'), 'Agent_Post')' successfully.
Data uploaded to collection 'Collection(Database(MongoClient(host=['ac-blk0g1e-shard-00-01.pbxvpvn.mongodb.net:27017', 'ac-blk0g1e-shard-00-02

In [22]:
#Create vector index for each collection, make sure you update the num_dimensions parameter with the correct value according to the embedding model you are using
create_vector_index("Agent_Post", num_dimensions=1536)
create_vector_index("Prompt_Engineering_Post", num_dimensions=1536)
create_vector_index("Adv_Attack_LLM_Post", num_dimensions=1536)

Vector search index created for collection 'Collection(Database(MongoClient(host=['ac-blk0g1e-shard-00-01.pbxvpvn.mongodb.net:27017', 'ac-blk0g1e-shard-00-02.pbxvpvn.mongodb.net:27017', 'ac-blk0g1e-shard-00-00.pbxvpvn.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='Cluster0', authsource='admin', replicaset='atlas-ue4svv-shard-0', tls=True), 'db'), 'Prompt_Engineering_Post')'.
Vector search index created for collection 'Collection(Database(MongoClient(host=['ac-blk0g1e-shard-00-01.pbxvpvn.mongodb.net:27017', 'ac-blk0g1e-shard-00-02.pbxvpvn.mongodb.net:27017', 'ac-blk0g1e-shard-00-00.pbxvpvn.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='Cluster0', authsource='admin', replicaset='atlas-ue4svv-shard-0', tls=True), 'db'), 'Adv_Attack_LLM_Post')'.


In [52]:
print(search_db("Agent_Post", "what are ai agents"))
print(Internet_search("what are ai agents?"))

Searching in collection 'Collection(Database(MongoClient(host=['ac-blk0g1e-shard-00-02.pbxvpvn.mongodb.net:27017', 'ac-blk0g1e-shard-00-01.pbxvpvn.mongodb.net:27017', 'ac-blk0g1e-shard-00-00.pbxvpvn.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='Cluster0', authsource='admin', replicaset='atlas-ue4svv-shard-0', tls=True), 'db'), 'Agent_Post')' for query: what are ai agents
Search results: [{'text': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#', 'score': 0.7318413257598877}, {'text': 'Agent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented 

In [42]:
tools = [
        {
            "type": "function",
            "function": {
                "name": "search_db",
                "description": "Get the data using vector search. Provide the name of the collection and input_query to perform vector search in the database.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "collection_name": {
                            "type": "string",
                            "description": "Name of the collection to perform vector search in.",
                        },
                        "input_query": {
                            "type": "string",
                            "description": "text to convert to embeddings and perform vector search in the database",
                        },
                    },
                    "required": ["collection_name", "input_query"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "Internet_search",
                "description": "Tool to perform internet search using duckduckgo.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "input": {"type": "string", "description": "Query to use while searching the internet"},
                    },
                    "required": ["input"],
                },
            },
        }
    ]

In [53]:
import os
import json
from openai import OpenAI
import dotenv
dotenv.load_dotenv()

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

def run_conversation(user_input):
    messages=user_input
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        tool_choice="auto",  
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    
    if tool_calls:
        available_functions = {
        "search_db": search_db,
        "Internet_search": Internet_search,
        }
        messages.append(response_message)  
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            if function_name == "search_db":
                function_response = function_to_call(
                    collection_name=function_args.get("collection_name"),
                    input_query=function_args.get("input_query"),
                )
            elif function_name == "Internet_search":
                function_response = function_to_call(
                    input=function_args.get("input"),
                )
                
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  
        second_response = client.chat.completions.create(model="gpt-4o-mini",messages= messages)
        output=second_response.choices[0].message.content
        return str(output)
    else:
        return str(response_message.content)
    
def main():
    messages = [
        {"role": "system", "content": """You are a smart AI Bot. Your task is to answer the user's query based on information from three relevant database collections:
            1. `Agent_Post`: Contains information on LLM Powered Autonomous Agents, including task decomposition, memory, and tool use, as well as case studies like 
                scientific discovery agents and generative agent simulations.
            2. `Prompt_Engineering_Post`: Contains detailed resources on prompt engineering, including techniques like zero-shot, few-shot, chain-of-thought prompting, 
                and automatic prompt design, as well as the use of external APIs and augmented language models.
            3. `Adv_Attack_LLM_Post`: Contains content on adversarial attacks on LLMs, including text generation, white-box vs black-box attacks, jailbreak prompting, 
                and various mitigation strategies.

            Attempt to search in the relevant database collection that matches the user's query. If no relevant information is found in the database, 
            perform an internal search. 
            Also perform Internet search to get realtime data.
            While responding, also mention where you sourced the data from.
         """},
        {"role": "assistant", "content": "How can I help you?"}
    ]
    
    print("AI Assistant: How can I help you? (Type 'quit' to exit)")
    
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() == 'quit':
            break
            
        messages.append({"role": "user", "content": user_input})
        response = run_conversation(messages)
        messages.append({"role": "assistant", "content": response})
        print("\nAI Assistant:", response, "\n")

main()

AI Assistant: How can I help you? (Type 'quit' to exit)
